# Introduction to Docker Compose
## Understanding how Docker Compose works

* Compose integrates with the Docker CLI as a plugin. 
* Compose interacts with Docker Engine through the API. 
* Compose provides a CLI and its actions translate into Docker Engine API calls. 
* Compose will read the Compose YAML ﬁle and generate resources accordingly. 
* Compose provides a layer for converting docker-compose commands into CLI-compliant ones. 
* Compose will interact with Docker objects and distinguish between them using labels.


e Docker CLI provides an API to create and load plugins. Once a plugin has been created and 
loaded on its invocation, the CLI command will be passed to it:

``` go
func pluginMain() { 
    plugin.Run(func(dockerCli command.Cli) *cobra.Command { 
        … 
    } 
} 

func main() { 
    if commands.RunningAsStandalone() { 
        os.Args = append([]string{"docker"}, compatibility.Convert(os.Args[1:])...) 
    } 
    pluginMain() 
}
```

Compose, being a plugin of the Docker CLI, will use a Docker Engine API client provided by the 
Docker CLI:

```go
lazyInit.WithService(compose.NewComposeService(dockerCli.Client(), dockerCli.ConfigFile()))
```

Each command passed to the Docker Compose plugin will lead to an interaction with the Docker Engine API on our host. For example, the internals of the ls command:

```go
func (s *composeService) List(ctx context.Context, opts api.ListOptions) ([]api.Stack, error) { 
    list, err := s.apiClient.ContainerList(ctx, moby.ContainerListOptions{ 
        Filters: filters.NewArgs(hasProjectLabelFilter()), 
        All: opts.All, 
    }) 
    if err != nil { 
        return nil, err 
    }
    return containersToStacks(list) 
} 
```